# Inter-Uni Datathon: Fraudulent Transaction Detection

#### Team Habaybi

---

## Part 1: Data Cleaning

In [41]:
# import required packages

import pandas as pd
import warnings 
import numpy as np

warnings.filterwarnings('ignore')


# Import Train and Test Datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')



In [42]:
train.describe()

,TransactionNumber,UserID,Age,NumDependents,Latitude,Longitude,UserTenure,IsFraud
count,10966.000000,10966.000000,10966.000000,10966.000000,10923.000000,10923.000000,10966.000000,10966.000000
mean,9117.500091,2483.885282,2751.845887,1.995714,-30.363255,141.254786,60.786157,0.364308
std,5293.669640,1429.402615,9730.988917,1.408035,6.962819,11.268395,34.254477,0.481257
min,1.000000,1.000000,-68.000000,0.000000,-41.640079,-112.026050,1.000000,0.000000
25%,4516.250000,1253.000000,26.000000,1.000000,-37.020100,142.702789,31.000000,0.000000
50%,9120.500000,2471.000000,34.000000,2.000000,-31.840233,144.964600,61.000000,0.000000
75%,13710.750000,3727.000000,43.000000,3.000000,-25.042261,145.612793,90.000000,1.000000
max,18277.000000,5000.000000,67000.000000,4.000000,57.851580,149.012375,119.000000,1.000000


In [43]:
# Show columns in dataset
train.columns

Index(['TransactionNumber', 'UserID', 'Age', 'Gender', 'Occupation',
       'EducationLevel', 'MaritalStatus', 'NumDependents', 'Income',
       'Expenditure', 'GiftsTransaction', 'TransactionDate', 'TransactionTime',
       'TransactionAmount', 'MerchantID', 'TransactionType',
       'TransactionLocation', 'DeviceType', 'Latitude', 'Longitude',
       'EmailDomain', 'Terrorism', 'UserTenure', 'IsFraud'],
      dtype='object')

##### Clean columns with Financial Data: Complete Currency Conversion to ensure data consistency

In [44]:

finance_cols = ['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount']
finance_train = train[['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount']]
finance_test = test[['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount']]

def clean_cash(data):
    """ 
    cleans all columns which contain financial data. Including conversion to AUD, then conversion to float datatype.
    """
    finance_cols = ['Income', 'Expenditure', 'GiftsTransaction', 'TransactionAmount']
    
    for col in finance_cols:
        
        data.columns = data.columns.str.replace(' ', '')
        data[col] = data[col].str.replace("AU$", "AUD", regex=False)
        data[col] = data[col].str.replace(" ", "", regex=False)
        data[f'{col}2'] = data[col]
        data[f'{col}Currency'] = data[col].apply(lambda x: 'AUD' if 'AUD' in str(x) else ('AED' if 'AED' in str(x) else ('GBP' if '£' in str(x) else ('GBP' if 'GBP' in str(x) else ''))))
        data[f'{col}2'] = data[f'{col}2'].str.replace("AUD", "", regex=False).str.strip()
        data[f'{col}2'] = data[f'{col}2'].str.replace("AED", "", regex=False).str.strip()
        data[f'{col}2'] = data[f'{col}2'].str.replace("£", "", regex=False).str.strip()
        data[f'{col}2'] = data[f'{col}2'].str.replace("GBP", "", regex=False).str.strip()
        data[f'{col}2'] = data[f'{col}2'].astype(float)
    
    data = data.drop(data.columns[:4], axis=1)
    data.columns = data.columns.str.replace('2', '')
    
    
    for col in finance_cols:
        data.loc[data[f'{col}Currency'] == 'AED', col] = data[col] * 0.41
        data.loc[data[f'{col}Currency'] == 'GBP', col] = data[col] * 1.96
    
    data = data[finance_cols]
  
    return data

cash_train_cleaned = clean_cash(finance_train)
cash_test_cleaned = clean_cash(finance_test)


In [45]:
train = train.drop(columns=finance_cols)
train = pd.concat([train, cash_train_cleaned], axis=1)

test = test.drop(columns=finance_cols)
test = pd.concat([test, cash_test_cleaned], axis=1)

##### Clean DeviceType Column

In [46]:
 # cleaning device column
items_to_be_grouped = ["mob", "galaxys7", "iphone 15", "android", "smartphone"]

for device in items_to_be_grouped:
    train['DeviceType'] = np.where(train['DeviceType'] == device, "Mobile", train["DeviceType"])
    test['DeviceType'] = np.where(test['DeviceType'] == device, "Mobile", test["DeviceType"])

##### Clean Age Column

In [47]:
# cleaning Age column
train['Age'] = train['Age'].apply(lambda x: abs(x) if x < 0 else x / 1000 if x > 100 else x)
test['Age'] = test['Age'].apply(lambda x: abs(x) if x < 0 else x / 1000 if x > 100 else x)

train['Age'] = train['Age'].astype(int)
test['Age'] = test['Age'].astype(int)

In [48]:

replace_female = ["Female", "fem", "she", "woman", "isnotmale"]
replace_male = ["Male", "he", "man", "isnotfemale"]

def fix_gender(data):
    for index, value in data["Gender"].items():
        if value in replace_female:
            data.at[index, "Gender"] = "Female"
        elif value in replace_male:
            data.at[index, "Gender"] = "Male"
        
train['Gender'] = fix_gender(train)
test['Gender'] = fix_gender(test)


##### One-Hot Encode Terrorism Column

In [49]:
# converting terrorism column from bool to int
train["Terrorism"] = train["Terrorism"].astype(int)
test["Terrorism"] = test["Terrorism"].astype(int)

##### Clean Location

In [50]:

replace_canberra = ["Canberra", "canberra", "CBR", "Cbr"]
replace_darwin = ["Darwin", "Drw", "DRW", "darwin"]
replace_adelaide = ["Adelaide", "Adl", "Adelaide City", "adl"]
replace_sydney = ["Sydney", "SYD", "Syd", "sydney"]
replace_hobart = ["Hobart", "hobart", "HBT", "Hbt"]
replace_brisbane = ["Brisbane", "BNE", "brisbane", "bne", "Bne"]
replace_perth = ["Perth", "perth", "PTH", "pth", "Pth"]
replace_melbourne = ["Melbourne", "Melb", "melbourne", "Mel", "MLB", "Melburn"]

def replace_location(column, replace_canberra, replace_darwin, replace_adelaide, replace_sydney, replace_hobart, replace_brisbane, replace_perth, replace_melbourne):
		replace_dict = {}
		replace_dict.update({value: "Canberra" for value in replace_canberra})
		replace_dict.update({value: "Darwin" for value in replace_darwin})
		replace_dict.update({value: "Adelaide" for value in replace_adelaide})
		replace_dict.update({value: "Sydney" for value in replace_sydney})
		replace_dict.update({value: "Hobart" for value in replace_hobart})
		replace_dict.update({value: "Brisbane" for value in replace_brisbane})
		replace_dict.update({value: "Perth" for value in replace_perth})
		replace_dict.update({value: "Melbourne" for value in replace_melbourne})
		return column.replace(replace_dict)

train['TransactionLocation'] = replace_location(train['TransactionLocation'], 
													replace_canberra, 
													replace_darwin, 
													replace_adelaide, 
													replace_sydney, 
													replace_hobart, 
													replace_brisbane, 
													replace_perth, 
													replace_melbourne)

test['TransactionLocation'] = replace_location(test['TransactionLocation'], 
													replace_canberra, 
													replace_darwin, 
													replace_adelaide, 
													replace_sydney, 
													replace_hobart, 
													replace_brisbane, 
													replace_perth, 
													replace_melbourne)

##### Clean Transaction Time 

In [51]:
# Function to clean time data
def clean_time_column(column):
    # First, replace any invalid separators like '/' with ':' 
    column = column.str.replace('/', ':', regex=False)

    # Try converting to datetime in 12-hour format first
    dt = pd.to_datetime(column, format='%I:%M:%S %p', errors='coerce')

    # If there are any NaT values, try converting to datetime in 24-hour format
    dt = dt.fillna(pd.to_datetime(column, format='%H:%M:%S', errors='coerce'))

    # Return as strings in 24-hour format
    return dt.dt.strftime('%H:%M:%S')

# Clean the 'Time' column
train['Cleaned_TransactionTime'] = clean_time_column(train['TransactionTime'])
test['Cleaned_TransactionTime'] = clean_time_column(test['TransactionTime'])

    

##### Divide Transaction Time into 4 different groups: Night, Morning, Afternoon and Night


In [52]:
# Define the start and end times for different parts of the day
night_start_time = pd.to_datetime('00:00:00').time()
night_end_time = pd.to_datetime('05:59:59').time()

morning_start_time = pd.to_datetime('06:00:00').time()
morning_end_time = pd.to_datetime('11:59:59').time()

afternoon_start_time = pd.to_datetime('12:00:00').time()
afternoon_end_time = pd.to_datetime('17:59:59').time()

evening_start_time = pd.to_datetime('18:00:00').time()
evening_end_time = pd.to_datetime('23:59:59').time()

# Convert the 'Cleaned_TransactionTime' column to datetime
train['Cleaned_TransactionTime'] = pd.to_datetime(train['Cleaned_TransactionTime'])
test['Cleaned_TransactionTime'] = pd.to_datetime(test['Cleaned_TransactionTime'])

# Function to categorize times of the day
def categorize_time_of_day(transaction_time):
    time_of_day = transaction_time.time()  # Extract time from datetime
    if morning_start_time <= time_of_day < morning_end_time:
        return 'Morning'
    elif afternoon_start_time <= time_of_day < afternoon_end_time:
        return 'Afternoon'
    elif evening_start_time <= time_of_day < evening_end_time:
        return 'Evening'
    else:
        return 'Night'

# Apply the categorization function to the 'Cleaned_TransactionTime' column
train['Transaction_Time_of_Day'] = train['Cleaned_TransactionTime'].apply(categorize_time_of_day)
train = train.drop(columns='Cleaned_TransactionTime')
test['Transaction_Time_of_Day'] = test['Cleaned_TransactionTime'].apply(categorize_time_of_day)
test = test.drop(columns='Cleaned_TransactionTime')


In [53]:
train = train.drop(columns={'EmailDomain', 'TransactionTime'})
test = test.drop(columns={'EmailDomain', 'TransactionTime'})

In [54]:
# exported cleaned data to csv
train.to_csv('train_cleaned.csv')
test.to_csv('test_cleaned.csv')